In [ ]:
# importing libraries
import numpy as np
import networkx as nx
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [ ]:
# Create a sample graph
G = nx.Graph()
G.add_edges_from([(1, 2), (2, 3), (3, 4), (4, 5), (5, 6)])  # Adding connections

# Assign sample text data to nodes
G.nodes[1]['text'] = "This product is amazing!"
G.nodes[2]['text'] = "this is bad."
G.nodes[3]['text'] = "It's not great."
G.nodes[4]['text'] = "I love it! Highly recommend."
G.nodes[5]['text'] = "i hate it."
G.nodes[6]['text'] = "everything is awesome."

In [ ]:
#pre processing data

# Extract text data from nodes
texts = [G.nodes[n]['text'] for n in G.nodes]

max_words = 5000

# Tokenization
tokenizer = Tokenizer(num_words = max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# Padding sequences
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding="post")


In [ ]:
#Build & Train an RNN (LSTM Model)python

# Define model parameters
embedding_dim = 16
output_dim = 1  # Binary classification (positive/negative)

# Create RNN Model
model = Sequential([
    Embedding(max_words, embedding_dim, input_length=max_length),
    LSTM(64, return_sequences=True),
    LSTM(32),
    Dense(16, activation="relu"),
    Dense(output_dim, activation="sigmoid")])  # Sigmoid for binary classification

# Compile Model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Dummy labels (1 for positive, 0 for negative)
import numpy as np
labels = [1, 0, 0, 1, 0, 1]
padded_sequences = np.array(padded_sequences)
labels = np.array(labels)

# Train Model
model.fit(padded_sequences, labels, epochs=20)


Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - accuracy: 0.6667 - loss: 0.6931
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - accuracy: 0.5000 - loss: 0.6926
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - accuracy: 0.6667 - loss: 0.6921
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 1.0000 - loss: 0.6916
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step - accuracy: 0.8333 - loss: 0.6911
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - accuracy: 0.6667 - loss: 0.6906
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - accuracy: 0.6667 - loss: 0.6900
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - accuracy: 0.6667 - loss: 0.6892
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - accuracy: 0.8333 - loss: 0.6884
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step - accuracy: 0.8333 - loss: 0.6874
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - accuracy: 1.0000 - loss: 0.6862
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step - accuracy: 1.0000 - loss: 0.684

In [ ]:
# Predict sentiment scores for each node
sentiment_scores = model.predict(padded_sequences)
for i, node in enumerate(G.nodes):
    G.nodes[node]['sentiment'] = float(sentiment_scores[i])  # Assigning sentiment score


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 609ms/step


<ipython-input-35-1165e659d541>:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  G.nodes[node]['sentiment'] = float(sentiment_scores[i])  # Assigning sentiment score


In [ ]:
# Analyze sentiment-based communities
positive_nodes = [n for n in G.nodes if G.nodes[n]['sentiment'] > 0.5]
negative_nodes = [n for n in G.nodes if G.nodes[n]['sentiment'] <= 0.5]

print("Positive Sentiment Nodes:", positive_nodes)
print("Negative Sentiment Nodes:", negative_nodes)

Positive Sentiment Nodes: [1, 4, 6]
Negative Sentiment Nodes: [2, 3, 5]
